In [10]:
# --- ЯЧЕЙКА ДИАГНОСТИКИ ---
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

try:
    # Используем НОВЫЙ, правильный base_url без /v1
    client = OpenAI(api_key=os.getenv("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")
    
    print("✅ Клиент DeepSeek успешно сконфигурирован с новым API.\n")
    print("Список доступных моделей:")
    
    models_list = client.models.list()
    for model in models_list:
        print(f"- {model.id}")

except Exception as e:
    print(f"❌ Ошибка при подключении или запросе списка моделей: {e}")

✅ Клиент DeepSeek успешно сконфигурирован с новым API.

Список доступных моделей:
- deepseek-chat
- deepseek-reasoner


In [11]:
# --- СКРИПТ ДЛЯ DEEPSEEK (ФИНАЛЬНАЯ РАБОЧАЯ ВЕРСИЯ) ---
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import requests # Мы используем requests напрямую, как вы и предложили
import base64

# --- Загрузка ключа ---
load_dotenv()
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

# --- Конфигурация с правильными данными ---
DEEPSEEK_API_URL = "https://api.deepseek.com/v1/vision" # <-- 1. ПРАВИЛЬНЫЙ URL
MODEL_DEEPSEEK = "deepseek-vision" # <-- 2. ПРАВИЛЬНАЯ МОДЕЛЬ
IMG_DIR = "img"
PROMPT = "Скажи, изображена ли на данном фото кошка породы \"Британская короткошерстная\", дай коротки ответ \"да\" или \"нет\"."
DEEPSEEK_RESULTS_FILE = "deepseek_vision_results.csv"

# --- Функция запроса, переписанная под новый API ---
def query_deepseek_vision(image_path, prompt):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    
    try:
        with open(image_path, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode('utf-8')
        
        # <-- 3. ПРАВИЛЬНАЯ СТРУКТУРА PAYLOAD
        payload = {
            "model": MODEL_DEEPSEEK,
            "image": base64_image,
            "prompt": prompt
        }

        response = requests.post(DEEPSEEK_API_URL, headers=headers, json=payload, timeout=60)
        response.raise_for_status() # Проверяем на ошибки HTTP (4xx/5xx)
        
        # Извлекаем ответ из JSON. Структура может отличаться, адаптируем под то, что вернет API.
        # Чаще всего это что-то вроде {'choices': [{'message': '...'}]}
        response_data = response.json()
        return response_data.get('choices', [{}])[0].get('message', 'Error: Could not parse response').strip()

    except requests.exceptions.RequestException as e:
        # Ловим сетевые ошибки и ошибки ответа сервера
        return f"Error: {type(e).__name__}, Details: {e.response.text if e.response else e}"
    except Exception as e:
        # Ловим другие возможные ошибки (например, при чтении файла)
        return f"Error: {type(e).__name__}, Details: {e}"

# --- Основной цикл (без изменений) ---
if DEEPSEEK_API_KEY:
    try:
        image_files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
        results_data = []
        
        print(f"Начинаем обработку {len(image_files)} изображений моделью DeepSeek...")
        for filename in tqdm(image_files, desc="DeepSeek Processing"):
            image_path = os.path.join(IMG_DIR, filename)
            response = query_deepseek_vision(image_path, PROMPT)
            results_data.append({"filename": filename, "deepseek_response": response})
            
        df_results = pd.DataFrame(results_data)
        df_results.to_csv(DEEPSEEK_RESULTS_FILE, index=False, encoding='utf-8')
        print(f"✅ Результаты DeepSeek сохранены в '{DEEPSEEK_RESULTS_FILE}'.")
        display(df_results.head())

    except FileNotFoundError:
        print(f"❌ Ошибка: Директория '{IMG_DIR}' не найдена.")
else:
    print("❌ API ключ для DeepSeek не найден в .env файле.")

Начинаем обработку 30 изображений моделью DeepSeek...


DeepSeek Processing: 100%|██████████| 30/30 [00:28<00:00,  1.05it/s]

✅ Результаты DeepSeek сохранены в 'deepseek_vision_results.csv'.


,filename,deepseek_response
0,brit1.jpg,"Error: HTTPError, Details: 404 Client Error: N..."
1,brit10.jpg,"Error: HTTPError, Details: 404 Client Error: N..."
2,brit2.jpg,"Error: HTTPError, Details: 404 Client Error: N..."
3,brit3.jpg,"Error: HTTPError, Details: 404 Client Error: N..."
4,brit4.jpg,"Error: HTTPError, Details: 404 Client Error: N..."
